In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [26]:
df6 = df6.apply(pd.to_numeric, errors = 'coerce')
df6.describe().iloc[:, 14:]
# 없는 Column = codesum, station_nbr, date,

,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,204684.000000,204906.000000,205572.000000,205572.000000,205128.000000,205572.000000,205572.000000,205572.000000,205572.000000,205572.0,205572.000000,205572.000000,205572.000000,205572.000000
mean,29.484393,30.005070,6.792657,16.206263,7.936364,0.520518,0.471922,0.129590,10.000000,6.0,56.000000,1.206375,3.005400,0.287257
std,0.160650,0.170558,4.022145,9.102250,3.696749,0.499580,0.499212,0.335852,3.000007,0.0,32.041718,9.748899,2.000268,0.452484
min,29.030000,29.530000,0.200000,1.000000,0.800000,0.000000,0.000000,0.000000,7.000000,6.0,1.000000,0.000000,0.000000,0.000000
25%,29.380000,29.890000,3.600000,12.000000,5.100000,0.000000,0.000000,0.000000,7.000000,6.0,28.000000,0.000000,1.000000,0.000000
50%,29.460000,29.970000,6.100000,17.000000,7.400000,1.000000,0.000000,0.000000,10.000000,6.0,56.000000,0.000000,3.000000,0.000000
75%,29.570000,30.090000,9.500000,19.000000,10.300000,1.000000,1.000000,0.000000,13.000000,6.0,84.000000,0.000000,5.000000,1.000000
max,30.110000,30.640000,19.000000,36.000000,19.400000,1.000000,1.000000,1.000000,13.000000,6.0,111.000000,283.000000,6.000000,1.000000


In [27]:
df6['store_nbr'].unique()

array([ 7, 13], dtype=int64)

In [28]:
df6_drop_columns = ['date', 'codesum', 'station_nbr']
df6 = df6.drop(columns = df6_drop_columns)

In [29]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df6_columns = df6.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df6_columns:
    if (i == 'resultdir'):
        df6[i].fillna(df6[i].mode()[0], inplace=True)
        print(df6[i].mode()[0])
    else:
        df6[i].fillna(df6[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df6['relative_humility'] = 100*(np.exp((17.625*((df6['dewpoint']-32)/1.8))/(243.04+((df6['dewpoint']-32)/1.8)))/np.exp((17.625*((df6['tavg']-32)/1.8))/(243.04+((df6['tavg']-32)/1.8))))

# 체감온도 계산
df6["windchill"] = 35.74 + 0.6215*df6["tavg"] - 35.75*(df6["avgspeed"]**0.16) + 0.4275*df6["tavg"]*(df6["avgspeed"]**0.16)

df6 = df6[df6['units'] != 0]

18


In [30]:
model_df6 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
sealevel + resultspeed + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df6)
result_df6 = model_df6.fit()

print(result_df6.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     1146.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:51:50   Log-Likelihood:                -3119.1
No. Observations:                5062   AIC:                             6296.
Df Residuals:                    5033   BIC:                             6486.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(nothing)[0]         -0.0322      0

In [31]:
model_df6 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + scale(preciptotal) + \
scale(sealevel) + scale(resultspeed) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df6)
result_df6 = model_df6.fit()

print(result_df6.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.864
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     1146.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:52:09   Log-Likelihood:                -3119.1
No. Observations:                5062   AIC:                             6296.
Df Residuals:                    5033   BIC:                             6486.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [33]:
anova_result_df6 = sm.stats.anova_lm(result_df6).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df6[anova_result_df6['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(preciptotal),1.0,0.839992,0.839992,4.159546,4.145262e-02
C(other),1.0,0.993676,0.993676,4.920575,2.658315e-02
scale(resultdir),1.0,1.418629,1.418629,7.024896,8.063572e-03
scale(resultspeed),1.0,1.631105,1.631105,8.077050,4.500972e-03
C(rain),1.0,2.180571,2.180571,10.797947,1.023088e-03
scale(relative_humility),1.0,2.730853,2.730853,13.522881,2.381310e-04
scale(tmin),1.0,2.867338,2.867338,14.198739,1.663620e-04
scale(heat),1.0,4.213073,4.213073,20.862675,5.051958e-06
C(week7),6.0,66.004005,11.000667,54.474091,2.054690e-65
C(item_nbr),7.0,6395.142505,913.591786,4524.005703,0.000000e+00


In [34]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df6.values, i) for i in range(df6.shape[1])]
vif["features"] = df6.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# item_nbr, store_nbr, preciptotal, resultdir, weekend, week7, preciptotal, other

,VIF Factor,features
0,1.113568,item_nbr
1,1.119178,store_nbr
2,1.151564,preciptotal
3,1.168552,resultdir
4,1.215726,snowfall
5,1.244349,units
6,1.356147,other
7,2.704121,depart
8,2.713628,week7
9,2.736350,weekend


In [38]:
# item_nbr, weekend, week7, preciptotal 
model_df6 = sm.OLS.from_formula('np.log1p(units) ~ C(item_nbr) + C(store_nbr) + scale(preciptotal) + scale(resultdir) + \
C(weekend) + C(week7) + C(other) + 0', data = df6)
result_df6 = model_df6.fit()

print(result_df6.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     1868.
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:57:49   Log-Likelihood:                -3147.4
No. Observations:                5062   AIC:                             6331.
Df Residuals:                    5044   BIC:                             6448.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(item_nbr)[5]         4.1237      0

In [39]:
X6 = df6[['week7', 'weekend', 'item_nbr', 'preciptotal', 'other', 'store_nbr', 'resultdir']]
y6 = df6['units']
model6 = LinearRegression()

cv6 = KFold(n_splits=10, shuffle=True, random_state=0)

cross_val_score(model6, X6, y6, scoring="r2", cv=cv6)

array([0.20319228, 0.20674801, 0.18206525, 0.21035294, 0.20105371,
       0.19092825, 0.1649597 , 0.14442243, 0.18766883, 0.1731834 ])